# Executive Summary
- The South region has most diversed cost versus pop_density
- The South region has, as the author Cindy Ermus wrote in her book,"The Gulf South, and the Gulf Coast in particular, is bound together by much more than geography or the shared experience of risk and vulnerability to wind, water, erosion, and biological exchanges,” she writes. “More fundamentally, the environment has helped define the region’s identity and largely determined its history, its social fabric, and its economy.” We can assumed, based on this graph that south region has significant high risk compare to other regions.
- The New York and California have the highest support_level. A seperated studies are recommended
- Climate change should be our national top concern, and higher amount of funding should be allocated.
- The KNN is our best model and we use it in out final test. The KNN model achived 90% accuracy

In [1]:
# import os and csv functions
import os
import csv
# import libaries
import pandas as pd
import numpy as np
from tabulate import tabulate
from datetime import datetime
from sklearn.impute import SimpleImputer
from tabula import read_pdf
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import math
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
# utilized for creating models and visualization
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# utilized for metrics on my models
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
# Homemade module
import prepare
# explore.py
import modeling

# Wrangle

In [2]:
# getting the file from local
df=prepare.get_fema_data()

In [3]:
# wrangle df
df=prepare.prepare_fema(df)

KeyError: 'stateabbrv'

In [ ]:
# check distribution
df.hist(figsize=(24, 10), bins=20)
plt.show()

- The majority of people has less than $8,000 per person
- The distribution graphs showing a lot outliers in this data. We decided not remove any outliers due to we would like all inclusive data for all states and counties.

In [ ]:
df.head()

In [ ]:
# split df into test (20%) and train_validate (80%)
tv_df, test = train_test_split(df, test_size=0.2, random_state=123)
# split train_validate off into train (70% of 80% = 56%) and validate (30% of 80% = 24%)
train, validate = train_test_split(tv_df, test_size=0.3, random_state=123)

# Wrangle Key Takeaway
- The data has 3142 rows, and 26 columns. This project specifically focusing on the Severe Storm, Draught and Hurricane.
- The project has included all 50 states' counties and the District of Columbia area. The risk score is retrived it from Federal Emergency Management Agency (FEMA). In our analysis we used three disasters: Drought, Severe Storm, and Hurricane.
- Next, we will explore the data and set our target variable==support level

# Exploration

In [ ]:
# lets make a map before out exploration
fig = px.choropleth(df,
                    locations='state', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='support_value',
                    color_continuous_scale="rdylbu", 
                    
                    )
fig.show()

### Takeaway
- The science.org research believes that the california will face the mega-flood in near future as climate change worsen
    - 'Recent evidence suggests that increases in western United States flood risk caused by anthropogenic warming may have been counteracted in recent decades by natural variability, but that further warming and shifts in natural variability will eventually “unmask” this accumulated increase in regional flood risk (51). Additional work suggests that the response of flood risk to climate change is likely to exhibit threshold behavior, at least in certain climatological and hydrological regimes (52), with a precipitation extremeness threshold dictating whether flood risk decreases (for smaller events, due to the antecedent soil aridification effect of warming temperatures) or increases (for the largest events, due to the overwhelming effect of large increases in precipitation intensity). Both of these considerations are especially germane to California' [science.org](https://www.science.org/doi/10.1126/sciadv.abq0995)
- The columbia study and New York Times journalist argued that the New York is not adequately prepared for the next storm like Hurricane Sandy.
    - 'The barrier debate comes as New York City is still struggling to respond to Sandy, and the larger need to carefully reshape an entire region’s infrastructure to adapt to climate change. In the more than seven years since the storm killed 72 people and caused 62 billion in damage, agencies have spent just 54 percent of the 14.7 billion allocated by the federal government to help the city recover and prepare for new storms.' [columbia.edu](https://news.climate.columbia.edu/2020/01/21/politics-cost-adapting-climate-change-new-york-city/)

### Q1: Whether the dense of the population contributed to the overall funding available or the risk level

In [ ]:
# set up th eenvironment
q1=tv_df.groupby('state',as_index=False).agg(sum)
q1.head()

In [ ]:
# bin the risk level to identify whether the funding available also associate with 
q1['risk_bin'] = pd.qcut(q1.risk_score, 4, labels=['low', 'median', 'hight', 'extremely_high'])

In [ ]:
# use catplot
sns.pointplot(data=q1, x="risk_bin", y="pop_density", dodge=True)

In [ ]:
# use catplot
sns.pointplot(data=q1, x="risk_bin", y="state_funding", dodge=True)

### q1 Key Takeaway
- As the risk level rise, the funding will be impacted by the pop_density
- The extremely high risk has highest funding when the pop_density is high
- It is worth to further explore which region it is belong to and what kind of the disaster contributed to increased funding

### q2 the cost of the disaster per region per risk level per pop_density

In [ ]:
# set up the environment
q2 = tv_df.copy()
q2['risk_bin'] = pd.qcut(q2.risk_score, 4, labels=['low', 'median', 'hight', 'extremely_high'])

# create the region based on the state's location
west = ['WA','OR','CA','ID','NV','MT','WY','UT','AZ','CO','NM']
midwest = ['ND','MN','WI','MI','SD','NE','KS','IA','MO','IL','IN','OH']
south = ['TX','OK','AR','LA','MS','TN','KY','AL','GA','FL','SC','NC','VA','WV','MD','DE','DC']
northeast = ['PA','NJ','NY','CT','MA','RI','VT','NH','ME']
west_dict = {i:"west" for i in west}
midwest_dict = {i:"midwest" for i in midwest}
south_dict = {i:"south" for i in south}
northeast_dict = {i:"northeast" for i in northeast}
d = {**west_dict, **midwest_dict, **south_dict, **northeast_dict}
q2['Region'] = q2['state'].map(d)

In [ ]:
g = sns.jointplot(data=q2, x="pop_density", y="cost", hue='Region')
g.plot_joint(sns.kdeplot, color="r", zorder=0, levels=6)
g.plot_marginals(sns.rugplot, color="r", height=-.15, clip_on=False)
plt.show()

### q2 Key Takeaway
- The South region has most diversed cost versus pop_density
- The South region has as the author [Cindy Ermus wrote](https://www.theatlantic.com/technology/archive/2017/08/why-the-gulf-coast-is-uniquely-vulnerable-to-disasters-hurricane-harvey/538374/) in her book,"The Gulf South, and the Gulf Coast in particular, is bound together by much more than geography or the shared experience of risk and vulnerability to wind, water, erosion, and biological exchanges,” she writes. “More fundamentally, the environment has helped define the region’s identity and largely determined its history, its social fabric, and its economy.”
- We can assumed, based on this graph that south region has significant high support value compare to other regions

### q3: Based on the q2 takeaway, we are going to make a hypothesis test that whether the south region has higher support value compare to the other regions.

In [ ]:
# set up the environment
q3 = tv_df.copy()

In [ ]:
# create the region based on the state's location
west = ['WA','OR','CA','ID','NV','MT','WY','UT','AZ','CO','NM']
midwest = ['ND','MN','WI','MI','SD','NE','KS','IA','MO','IL','IN','OH']
south = ['TX','OK','AR','LA','MS','TN','KY','AL','GA','FL','SC','NC','VA','WV','MD','DE','DC']
northeast = ['PA','NJ','NY','CT','MA','RI','VT','NH','ME']
west_dict = {i:"west" for i in west}
midwest_dict = {i:"midwest" for i in midwest}
south_dict = {i:"south" for i in south}
northeast_dict = {i:"northeast" for i in northeast}
d = {**west_dict, **midwest_dict, **south_dict, **northeast_dict}
q3['Region'] = q3['state'].map(d)

# Hypothesis

- H0: South Region's risk score equals or less than non-south region risk score
- Ha: South Region's risk score greater than non-south region risk score
- Significance Level α is already set to .05


In [ ]:
# Set up the environment
south_r = q3[q3.Region == 'south'].support_value
non_south= q3[q3.Region != 'south'].support_value
west=q3[q3.Region == 'west'].support_value
midwest=q3[q3.Region == 'midwest'].support_value
northeast=q3[q3.Region == 'northeast'].support_value

In [ ]:
# check variance
print(south_r.var())
print(non_south.var())

In [ ]:
# put into the test
t, p = stats.ttest_ind(south_r, non_south, equal_var=False)
t, p
p<0.05

### We reject the null value, that the sourth region's risk score is greater than other regions

In [ ]:
stats.kruskal(south, west,northeast,midwest)

### Using Kruskal-Wallis test, non-parametric test for ANOVA, also shows us that the mean of Risk Score  from the 4 Regions is significantly different.

### Q3 takeaway
- The south region risk score is higher than other regions
- The Kruskal-Wallis test also concur that the risk score is significantly different among 4 regions
- We can further assumed the supporting level in south region is higher than other region 

### q4: Based on the q3 takeaway, we are going to make a hypothesis test that whether the south region has higher supporting level compare to the other regions based on their risk_level.

In [ ]:
# Set up environment
q4 = tv_df.copy()
q4['risk_level'] = pd.qcut(q4.risk_score, 4, labels=['low', 'median', 'hight', 'extremely_high'])
# create the region based on the state's location
west = ['WA','OR','CA','ID','NV','MT','WY','UT','AZ','CO','NM']
midwest = ['ND','MN','WI','MI','SD','NE','KS','IA','MO','IL','IN','OH']
south = ['TX','OK','AR','LA','MS','TN','KY','AL','GA','FL','SC','NC','VA','WV','MD','DE','DC']
northeast = ['PA','NJ','NY','CT','MA','RI','VT','NH','ME']
west_dict = {i:"west" for i in west}
midwest_dict = {i:"midwest" for i in midwest}
south_dict = {i:"south" for i in south}
northeast_dict = {i:"northeast" for i in northeast}
d = {**west_dict, **midwest_dict, **south_dict, **northeast_dict}
q4['Region'] = q4['state'].map(d)

H0: support level is not depended on the risk level

In [ ]:
# create observed dataframe
observed = pd.crosstab(q4.risk_level, q4.Region)
observed

In [ ]:
# chi2 test
chi2, p, degf, expected = stats.chi2_contingency(observed)

print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p}')
print('P is less than Significance Level:',p<0.05)

### Q4 key takeawat
- The support level is depended on the risk score

# Exploration Key takeaway
- The south region has higher support level in average compare to the other regions
- The risk level is associated with support level

# Modeling

In [ ]:
# split the data and set up the environment for our scaling
X_train, y_train, X_validate, y_validate, X_test, y_test = modeling.train_validate_test(df, 'support_level')

In [ ]:
# scale the data for the modeling
X_train_scaled, X_validate_scaled, X_test_scaled = modeling.scale_data(X_train, X_validate, X_test)

In [ ]:
# find out the baseline for the model
(y_train == 'bottom tier').mean()

In [ ]:
# Setting up my baseline accuracy.  Models must have a better accuracy than this established baseline.
print(f'The baseline accuracy for bottom tier counties in all cases within the Climate Risk Assessment dataset is {(y_train == "bottom tier").mean():.2}')

In [ ]:
# use created module to train with different models and find out which model has best accuracy.
modeling.modeling(X_train_scaled, y_train, X_validate_scaled, y_validate, 5, 3, 15)

### Key takeaway
- The Kneighbors Classifier has the highest accuracy when we have it with max_depth=3 and random_state=123
- We will going to use the KNN model for our final test

In [ ]:
# putting KNN into final test
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_scaled, y_train)
pred_test = knn.predict(X_test_scaled)
print('KNN')
print('')
print('test score: ')
print(classification_report(y_test, pred_test))

In [ ]:
# create a confusion matrix
actual_test=y_test.copy()
ConfusionMatrixDisplay(confusion_matrix(actual_test,pred_test),display_labels=knn.classes_).plot()
plt.show()

In [ ]:
# predict the dataset with our random forest model
prob_test = knn.predict_proba(X_test)

In [ ]:
# create a probability dataframe
prob_df = pd.DataFrame(prob_test, columns=knn.classes_.tolist())
prob_df.head()

In [ ]:
# reset the indext
reset_test =test.reset_index()
reset_test.head()

In [ ]:
# put two dataframe together
test_prob_df = pd.concat([reset_test, prob_df], axis=1)
test_prob_df.head()

In [ ]:
# use the propability result to create a column call predicted
test_prob_df['predicted'] = pred_test
test_prob_df.head()

In [ ]:
# retrive columns of interest
# create the region based on the state's location
west = ['WA','OR','CA','ID','NV','MT','WY','UT','AZ','CO','NM']
midwest = ['ND','MN','WI','MI','SD','NE','KS','IA','MO','IL','IN','OH']
south = ['TX','OK','AR','LA','MS','TN','KY','AL','GA','FL','SC','NC','VA','WV','MD','DE','DC']
northeast = ['PA','NJ','NY','CT','MA','RI','VT','NH','ME']
west_dict = {i:"west" for i in west}
midwest_dict = {i:"midwest" for i in midwest}
south_dict = {i:"south" for i in south}
northeast_dict = {i:"northeast" for i in northeast}
d = {**west_dict, **midwest_dict, **south_dict, **northeast_dict}
test_prob_df['Region'] = test_prob_df['state'].map(d)
csv_df = test_prob_df[['support_level', 'Region','predicted']]

In [ ]:
# check the dataframe
csv_df

### Modeling Takeaway
- The KNN is our best model and we use it in out final test
- The KNN model achived 90% accuracy

# Recommendation, Conclusion, and Next Step

## Recommendation
- New York and California have the highest support_level. A separate study is recommended for these specific states and their counties within.
- Climate change should be our top concern at the all levels of government, and higher amount of funding should be allocated accordingly.
- The KNN is our best model in our training, so we used it in our test. The model achived 90% accuracy 

## Conclusion 
- The initial data indicated that the New York and California have the highest risk in the United States in terms of funding and studies were found to back up those findings.
- The South region has the most diverse cost in damages versus pop_density
- The South region has, as the author Cindy Ermus wrote in her book,"The Gulf South, and the Gulf Coast in particular, is bound together by much more than geography or the shared experience of risk and vulnerability to wind, water, erosion, and biological exchanges,” she writes. “More fundamentally, the environment has helped define the region’s identity and largely determined its history, its social fabric, and its economy.” We can assumed, based on this graph that south region has significant high risk compare to other regions

## Next Step
- Expand the scope of the data (include more hazards and possibly more geospatial data)
- Improve the machine learning model by using other algorithms
- Conduct a seperate study regarding New York and California or possibly introduce a model that can look at specific regions at any given time according to the most common hazards found in the area.
